# 02 — Model Training (MVP)

This notebook trains a **small, interpretable regression model** to predict **daily kWh demand**.

- v1 uses **synthetic labels** (we don’t have real per-site utilization yet).
- The goal is to demonstrate an end-to-end ML workflow: features → model → evaluation → export.

The exported artifact is a **JSON file** (weights + intercept) used by the FastAPI `POST /api/predict` endpoint.


In [ ]:
import json
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

FEATURES = [
    "traffic_index",
    "pop_density_index",
    "renters_share",
    "income_index",
    "poi_index",
]

DATA = Path("../data/processed/sites_worcester.json")

if DATA.exists():
    df = pd.read_json(DATA)
    X = df[FEATURES].to_numpy()
    # v1 label: use the heuristic kWh estimate as a stand-in (plus noise)
    y = df["daily_kwh_estimate"].to_numpy()

    rng = np.random.default_rng(7)
    y = y + rng.normal(0.0, 12.0, size=len(y))
else:
    # fallback: fully synthetic dataset
    rng = np.random.default_rng(7)
    X = np.column_stack(
        [
            rng.beta(2.0, 2.0, size=3000),
            rng.beta(2.2, 2.5, size=3000),
            rng.beta(2.0, 3.0, size=3000),
            rng.beta(2.5, 2.0, size=3000),
            rng.beta(2.0, 2.2, size=3000),
        ]
    )

    traffic, pop, renters, income, poi = X.T
    expected_sessions = 4.0 + 8.0 * traffic + 6.0 * pop + 1.5 * poi + 1.0 * renters - 0.5 * income
    y = expected_sessions * 25.0 + rng.normal(0.0, 12.0, size=len(expected_sessions))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

model = Ridge(alpha=1.0, random_state=7)
model.fit(X_train, y_train)

pred = model.predict(X_test)
rmse = float(np.sqrt(mean_squared_error(y_test, pred)))

rmse

In [ ]:
# Feature weights (interpretable linear model)
weights = pd.Series(model.coef_, index=FEATURES).sort_values(key=lambda s: s.abs(), ascending=False)
weights

In [ ]:
# Export a JSON artifact consumed by the FastAPI backend
artifact = {
    "model_type": "ridge",
    "features": FEATURES,
    "coef": [float(x) for x in model.coef_.tolist()],
    "intercept": float(model.intercept_),
    "trained_on": "synthetic (heuristic label)",
    "rmse": float(rmse),
}

out = Path("../backend/models/site_demand_model.json")
out.parent.mkdir(parents=True, exist_ok=True)
out.write_text(json.dumps(artifact, indent=2))

out